In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
from pprint import pprint
import requests

Import the required libraries. BeutifulSoup will be used as the parser. Import requests is required to make HTTP request from URL's.

In [2]:
headers = {
    'Host': 'www.imdb.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
}

The host website I will be parsing is IMDB.com. The user agent shows that my browser is compatible with Mozilla/5.0 and uses windows 10. I defined a function called get_title_details, that takes an IMDb ID as an input and uses it to parse IMDb and return details about the movie with the associated IMDb id. It accomplishes this by sending a HTTP get request to the hosts url, which is IMDb in this case. Using the request library. It will return details on budget, box_office, directors, etc.

In [3]:
def get_title_details(imdb_id):
    imdb_url = f'https://www.imdb.com/title/{imdb_id.strip()}/'

    page = bs(requests.get(imdb_url, headers=headers).text, 'lxml')
    
    return {
        'Budget': page.find('li', {'data-testid': 'title-boxoffice-budget'}).find('div').text if page.find('li', {'data-testid': 'title-boxoffice-budget'}) else '',
        'Box Office US/CA': page.find('li', {'data-testid': 'title-boxoffice-grossdomestic'}).find('div').text if page.find('li', {'data-testid': 'title-boxoffice-grossdomestic'}) else '',
        'Box Office Worldwide': page.find('li', {'data-testid': 'title-boxoffice-cumulativeworldwidegross'}).find('div').text if page.find('li', {'data-testid': 'title-boxoffice-cumulativeworldwidegross'}) else '',
        'Length': page.find('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt'}).find_all('li')[-1].text if page.find('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt'}) else '',
        'Rating': page.find('div', {'data-testid': 'hero-rating-bar__aggregate-rating__score'}).find('span').text if page.find('div', {'data-testid': 'hero-rating-bar__aggregate-rating__score'}) else '',
        'Director Names': ', '.join([director.text.strip() for director in page.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})[0].find_all('li')]) if len(page.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})) > 1 else '',
        'Director Links': ['https://www.imdb.com' + director.find('a').get('href').split('?')[0] for director in page.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})[0].find_all('li')] if len(page.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})) > 1 else '',
        'Cast Names': ', '.join([cast.text.strip() for cast in page.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})[-1].find_all('li')]) if len(page.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})) > 0 else '',
        'Cast Links': ['https://www.imdb.com' + director.find('a').get('href').split('?')[0] for director in page.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})[-1].find_all('li')] if len(page.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})) > 0 else '',
    }

Below I haved defined a function called get_title_awards, that takes an IMDb ID as an input and uses it to parse IMDb and return details about the movies awards.

In [4]:
def get_title_awards(imdb_id):
    imdb_url = f'https://www.imdb.com/title/{imdb_id}/awards/'
    page = bs(requests.get(imdb_url, headers=headers).text, 'lxml')
    
    return ', '.join([award.find_all('td')[-1].text.strip() for award in page.find('table', {'class': 'awards'}).find_all('tr')]) if page.find('table', {'class': 'awards'}) else ''

This code reads a CSV file named "remade_imdb.csv" into a pandas DataFrame, df. This file stores all the original movies imdb links and the remade movies imdb links. After the csv file is read a list named new_columns_to_add is defined. This list contains the names of new columns that should be added to the dataframe. The code loops through every name in the list, checking if the column corresponding to that name exists in the dataframe. 

If the column doesn't exist that column is then added to the dataframe with the corresponding name taken from the list. This is completed using the df.insert method. The new column has empty strings for every row, this is accomplished using len(df).

Then, it defines a list named new_columns_to_add that contains the names of new columns that should be added to the DataFrame.
This code gives new columns to the DataFrame with default values, provided these columns don't already exist.

In [5]:
df = pd.read_csv('remade_imdb.csv')

new_columns_to_add = [
    'Original Budget',
    'Original Box Office US/CA',
    'Original Box Office Worldwide',
    'Original Length',
    'Original Rating',
    'Original Director Names',
    'Original Director Links',
    'Original Cast Names',
    'Original Cast Links',
    'Original Awards',
    'Remade Budget',
    'Remade Box Office US/CA',
    'Remade Box Office Worldwide',
    'Remade Length',
    'Remade Rating',
    'Remade Director Names',
    'Remade Director Links',
    'Remade Cast Names',
    'Remade Cast Links',
    'Remade Awards',
]

for column in new_columns_to_add:
    if not column in df.columns:
        df.insert(loc=len(df.columns), column=column, value=['']*len(df), allow_duplicates=True)

In [7]:
for x in range(len(df)):
    oIMDb_id = df.iloc[x]['oIMDb']
    original_details = get_title_details(oIMDb_id)

    rIMDb_id = df.iloc[x]['rIMDb']
    remade_details = get_title_details(rIMDb_id)

    for key, value in original_details.items():
        df.at[x, f'Original {key}'] = value

    for key, value in remade_details.items():
        df.at[x, f'Remade {key}'] = value

    df.at[x, 'Original Awards'] = get_title_awards(oIMDb_id)
    df.at[x, 'Remade Awards'] = get_title_awards(rIMDb_id)


    print(df.loc[x])
    df = df.fillna('')

    df.to_csv('RemadeOutput.csv', index=None)

oIMDb                                                                    tt0057212
rIMDb                                                                    tt1436045
Original Budget                                                                   
Original Box Office US/CA                                                         
Original Box Office Worldwide                                                     
Original Length                                                              2h 5m
Original Rating                                                                7.4
Original Director Names                                                Eiichi Kudô
Original Director Links                     [https://www.imdb.com/name/nm0473791/]
Original Cast Names              Takayuki Akutagawa, Chiezô Kataoka, Kôtarô Satomi
Original Cast Links              [https://www.imdb.com/name/nm3321261/, https:/...
Original Awards                                                                   
Rema

oIMDb                                                                    tt0057809
rIMDb                                                                    tt0096977
Original Budget                                                                   
Original Box Office US/CA                                                         
Original Box Office Worldwide                                                     
Original Length                                                             1h 55m
Original Rating                                                                7.3
Original Director Names                                              George Seaton
Original Director Links                     [https://www.imdb.com/name/nm0780833/]
Original Cast Names                      James Garner, Eva Marie Saint, Rod Taylor
Original Cast Links              [https://www.imdb.com/name/nm0001258/, https:/...
Original Awards                                                                   
Rema

oIMDb                                                                    tt0054594
rIMDb                                                                    tt0119137
Original Budget                                             $2,000,000 (estimated)
Original Box Office US/CA                                                         
Original Box Office Worldwide                                                     
Original Length                                                             1h 32m
Original Rating                                                                6.7
Original Director Names                                           Robert Stevenson
Original Director Links                     [https://www.imdb.com/name/nm0829038/]
Original Cast Names                       Fred MacMurray, Nancy Olson, Keenan Wynn
Original Cast Links              [https://www.imdb.com/name/nm0534045/, https:/...
Original Awards                  Best Cinematography, Black-and-White\nEdward C...
Rema

oIMDb                                                                    tt0078767
rIMDb                                                                    tt0384806
Original Budget                                             $4,700,000 (estimated)
Original Box Office US/CA                                              $86,432,000
Original Box Office Worldwide                                          $86,432,000
Original Length                                                             1h 57m
Original Rating                                                                6.1
Original Director Names                                           Stuart Rosenberg
Original Director Links                     [https://www.imdb.com/name/nm0742341/]
Original Cast Names                       James Brolin, Margot Kidder, Rod Steiger
Original Cast Links              [https://www.imdb.com/name/nm0000981/, https:/...
Original Awards                          Best Music, Original Score\nLalo Schifrin
Rema

oIMDb                                                                    tt0082479
rIMDb                                                                    tt0202233
Original Budget                                                                   
Original Box Office US/CA                                                         
Original Box Office Worldwide                                                     
Original Length                                                             1h 32m
Original Rating                                                                5.3
Original Director Names                                                Joe D'Amato
Original Director Links                     [https://www.imdb.com/name/nm0001090/]
Original Cast Names                    Tisa Farrow, Saverio Vallone, Serena Grandi
Original Cast Links              [https://www.imdb.com/name/nm0268533/, https:/...
Original Awards                                                                   
Rema

...

oIMDb                                                                    tt0207201
rIMDb                                                                    tt0207201
Original Budget                                            $70,000,000 (estimated)
Original Box Office US/CA                                             $182,811,707
Original Box Office Worldwide                                         $374,111,707
Original Length                                                              2h 7m
Original Rating                                                                6.4
Original Director Names                                               Nancy Meyers
Original Director Links                     [https://www.imdb.com/name/nm0583600/]
Original Cast Names                           Mel Gibson, Helen Hunt, Marisa Tomei
Original Cast Links              [https://www.imdb.com/name/nm0000154/, https:/...
Original Awards                                                  Best Fantasy Film
Rema

oIMDb                                                                    tt0032128
rIMDb                                                                    tt0041341
Original Budget                                                                   
Original Box Office US/CA                                                         
Original Box Office Worldwide                                                     
Original Length                                                             1h 20m
Original Rating                                                                6.2
Original Director Names                                             Gregory Ratoff
Original Director Links                     [https://www.imdb.com/name/nm0711855/]
Original Cast Names                    Loretta Young, Warner Baxter, Binnie Barnes
Original Cast Links              [https://www.imdb.com/name/nm0949835/, https:/...
Original Awards                                                                   
Rema

oIMDb                                                                    tt0032138
rIMDb                                                                    tt2027255
Original Budget                                             $2,777,000 (estimated)
Original Box Office US/CA                                              $24,668,669
Original Box Office Worldwide                                          $25,637,669
Original Length                                                             1h 42m
Original Rating                                                                8.1
Original Director Names                 Victor Fleming, George Cukor, Mervyn LeRoy
Original Director Links          [https://www.imdb.com/name/nm0281808/, https:/...
Original Cast Names                         Judy Garland, Frank Morgan, Ray Bolger
Original Cast Links              [https://www.imdb.com/name/nm0000023/, https:/...
Original Awards                  Best Music, Original Song\nHarold Arlen\n(musi...
Rema

oIMDb                                                                    tt0030989
rIMDb                                                                    tt0087866
Original Budget                                                                   
Original Box Office US/CA                                                         
Original Box Office Worldwide                                                     
Original Length                                                             1h 42m
Original Rating                                                                6.6
Original Director Names                                                Jack Conway
Original Director Links                     [https://www.imdb.com/name/nm0176699/]
Original Cast Names                  Robert Taylor, Vivien Leigh, Lionel Barrymore
Original Cast Links              [https://www.imdb.com/name/nm0001791/, https:/...
Original Awards                                                                   
Rema

oIMDb                                                                    tt0063829
rIMDb                                                                    tt0443295
Original Budget                                             $2,500,000 (estimated)
Original Box Office US/CA                                                         
Original Box Office Worldwide                                                     
Original Length                                                             1h 51m
Original Rating                                                                7.1
Original Director Names                                         Melville Shavelson
Original Director Links                     [https://www.imdb.com/name/nm0789520/]
Original Cast Names                         Lucille Ball, Henry Fonda, Van Johnson
Original Cast Links              [https://www.imdb.com/name/nm0000840/, https:/...
Original Awards                  Best Motion Picture - Comedy or Musical, Best ...
Rema

This code loops through each row in a Pandas DataFrame df. For each row, it extracts the oIMDb and rIMDb values and calling the get_title_details() function in order to obtain the data features for both orginal and remade movies. Each key value pair in the dictionaries are returned using get_title_details() for the original and remade movies, updating the corresponding columns in the current row of the DataFrame df.

The get_title_awards() function is called on the oIMDb and rIMDb values, updating the Original Awards and Remade Awards columns in the current row.The code will print the current row of the datagrame after each loop teration.The DataFrame df is filled with empty values for any missing values and written to a CSV file named Output Sample.csv without row indices.